In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt

In [17]:
# Grab with regular SHIPS

# Variables from SHIPS. HSTA and HEND are variables I made up for hours after RI start and hours before RI end.
variables = ['VMAX','SHRD','SHTD','VMPI','RSST','RHLO','IR00','DTL','HSTA','HEND','RI_Number']

basin = 'north_atlantic' # east_pacific or north_atlantic

# Read in RI cases
RI = pd.read_csv('/Users/acheung/data/RI_Cases/RI_Events_'+basin+'.csv')

# Convert columns to datetime format
RI["RI Start"] = pd.to_datetime(RI["RI Start"])
RI["RI End"] = pd.to_datetime(RI["RI End"])

list_pds = [] # Saves all the information as one big list!

for var_ind in range(len(variables)): # Loops through all variables
    if (variables[var_ind] == 'HSTA') | (variables[var_ind] == 'HEND') | (variables[var_ind] == 'RI_Number'): # For time-hour variables
        SHIPS_data = pd.read_csv('/Users/acheung/data/SHIPS/SHIPS_'+variables[0]+'_'+basin+'.csv') # just pick another variable to get time elements
    else:
        SHIPS_data = pd.read_csv('/Users/acheung/data/SHIPS/SHIPS_'+variables[var_ind]+'_'+basin+'.csv')

    SHIPS_data['Time'] = pd.to_datetime(SHIPS_data['Time'])

    all_data_arrays = []
    all_storm_ids = []
    all_storm_times = []
    all_storm_names = []
    t_start_list = []
    t_end_list = []

    for i in range(len(RI)):
        # Grab current storm data
        current_storm_data = SHIPS_data.where(SHIPS_data['Storm_ID'] ==
                                                RI.iloc[[i]]['Storm_ID'].values[0]).dropna()
        current_storm_data_indexed = current_storm_data.set_index('Time')
        current_storm_data_indexed.index = pd.to_datetime(current_storm_data_indexed.index)
        # We call shear for 24 hours before RI, but sometimes we dont have data back that far if it was not a TC oficially
        
        # Grab only times interested from RI (24 h before) to RI end
        # Sometimes due to data availability or storm intensity, we may get less than this time period
        data_array = current_storm_data_indexed.loc[(RI.iloc[
            [i]]['RI Start']+timedelta(hours = -24)).values[0]:RI.iloc[[i]]['RI End'].values[0]]
        
        # Save storm IDs
        all_storm_ids.append(data_array['Storm_ID'].values)
        
        # Save storm name
        all_storm_names.append(np.repeat(RI.iloc[[i]]['Storm_Name'].values[0],data_array.shape[0]))
        
        # Save datetime
        all_storm_times.append(data_array.index.values)
        
        if variables[var_ind] == 'HSTA': # for appending start time variable
            time_after_start = data_array.index - RI.iloc[[i]]['RI Start'].values[0]
            all_data_arrays.append(time_after_start)
            
        elif variables[var_ind] == 'HEND': # for appending end time variable
            time_before_end = data_array.index - RI.iloc[[i]]['RI End'].values[0]
            all_data_arrays.append(time_before_end)
            
        elif variables[var_ind] == 'RI_Number':
            curr_RI_num = RI.iloc[i]['RI_Number']
            RI_num_arr = np.repeat(curr_RI_num,data_array.shape[0])
            all_data_arrays.append(RI_num_arr)

        else: # Append data for when not time-data variables
            all_data_arrays.append(data_array[variables[var_ind]].values)

    # Creates a pandas dataframe with datetime and atcf id as axis for an individual variable
    alldata_pandad = pd.DataFrame(np.concatenate(all_data_arrays),
                              index=[np.concatenate(all_storm_ids),np.concatenate(all_storm_names),
                                     np.concatenate(all_storm_times)],columns=[variables[var_ind]])
    
    # Save all pandas dataframes of each variable with the same axis to a big list!
    list_pds.append(alldata_pandad)

# Concatenate all the pandas arrays
SHIPS_concat = pd.concat(list_pds,axis=1)

# Rename the axis
SHIPS_concat = SHIPS_concat.rename_axis(('Storm_ID','Storm_Names','Time'))

# Save to CSV
SHIPS_concat.to_csv("/Users/acheung/data/RI_Cases/RI_Events_with_SHIPS_Data_"+basin+".csv")


In [18]:
pd.set_option('display.max_rows', None)
SHIPS_concat

VMAX  SHRD   SHTD   VMPI  RSST  \
Storm_ID Storm_Names Time                                                   
AL032000 ALBERTO     2000-08-10 12:00:00   65.0   0.9  244.0  143.0  28.4   
                     2000-08-10 18:00:00   65.0   5.6  294.0  142.0  28.5   
                     2000-08-11 00:00:00   70.0   3.1  262.0  141.0  28.6   
                     2000-08-11 06:00:00   75.0   8.8  186.0  142.0  28.6   
                     2000-08-11 12:00:00   80.0   9.4  188.0  137.0  28.3   
                     2000-08-11 18:00:00   85.0   8.7  206.0  127.0  27.9   
                     2000-08-12 00:00:00   90.0   9.2  175.0  122.0  27.7   
                     2000-08-12 06:00:00  100.0  14.2  148.0  116.0  27.3   
                     2000-08-12 12:00:00  110.0  19.9  155.0  115.0  27.1   
AL072000 DEBBY       2000-08-19 18:00:00   30.0   0.8  355.0  127.0  27.7   
                     2000-08-20 00:00:00   30.0   2.5   10.0  127.0  27.7   
                     2000-08-20 06:00:00   35.0  10.3   62.0  126.0  27.6   
                     2000-08-20 12:00:00   40.0  13.1   84.0  122.0  27.5   
                     2000-08-20 18:00:00   45.0   8.9   93.0  125.0  27.6   
                     2000-08-21 00:00:00   55.0  14.4   64.0  127.0  27.7   
                     2000-08-21 06:00:00   65.0  14.5   52.0  130.0  27.9   
                     2000-08-21 12:00:00   75.0  13.2   79.0  136.0  28.3   
AL102000 FLORENCE    2000-09-10 18:00:00   30.0   6.9  151.0  126.0  27.7   
                     2000-09-11 00:00:00   30.0   8.1  128.0  128.0  27.7   
                     2000-09-11 06:00:00   30.0  15.3  131.0  130.0  27.7   
                     2000-09-11 12:00:00   45.0  18.0  161.0  134.0  27.8   
                     2000-09-11 18:00:00   65.0  12.0  165.0  133.0  28.0   
AL112000 GORDON      2000-09-14 12:00:00   25.0   7.8   87.0  140.0  28.8   
                     2000-09-14 18:00:00   25.0   7.3   90.0  133.0  28.7   
                     2000-09-15 00:00:00   25.0   9.6   90.0  131.0  28.7   
                     2000-09-15 06:00:00   25.0  14.4  111.0  135.0  28.7   
                     2000-09-15 12:00:00   25.0  17.9  132.0  136.0  28.6   
                     2000-09-15 18:00:00   30.0  10.7  127.0  131.0  28.7   
                     2000-09-16 00:00:00   40.0  17.7  103.0  138.0  29.0   
                     2000-09-16 06:00:00   50.0  25.7  103.0  143.0  29.1   
                     2000-09-16 12:00:00   55.0  21.7  108.0  152.0  29.2   
                     2000-09-16 18:00:00   60.0  19.7   87.0  156.0  29.4   
AL122000 HELENE      2000-09-19 18:00:00   30.0   7.5  196.0  149.0  29.6   
                     2000-09-20 00:00:00   30.0   6.4  186.0  147.0  29.5   
                     2000-09-20 06:00:00   25.0   8.7  163.0  152.0  29.6   
                     2000-09-20 12:00:00   25.0  14.3  153.0  154.0  29.6   
                     2000-09-20 18:00:00   25.0  12.6  136.0  141.0  29.4   
                     2000-09-21 00:00:00   25.0  15.0  130.0  143.0  29.3   
                     2000-09-21 06:00:00   35.0  19.9  125.0  144.0  29.3   
                     2000-09-21 12:00:00   45.0  23.8  126.0  145.0  29.2   
                     2000-09-21 18:00:00   60.0  24.9  124.0  146.0  29.1   
AL132000 ISAAC       2000-09-21 18:00:00   30.0  10.4   30.0  144.0  28.4   
                     2000-09-22 00:00:00   35.0   7.6   63.0  145.0  28.4   
                     2000-09-22 06:00:00   40.0   9.8   94.0  145.0  28.3   
                     2000-09-22 12:00:00   45.0   9.4   86.0  141.0  28.0   
                     2000-09-22 18:00:00   45.0   6.8   49.0  137.0  27.8   
                     2000-09-23 00:00:00   50.0   8.5   49.0  136.0  27.7   
                     2000-09-23 06:00:00   55.0   7.3   33.0  132.0  27.5   
                     2000-09-23 12:00:00   70.0   5.6   63.0  130.0  27.3   
                     2000-09-23 18:00:00   85.0   3.0  356.0  127.0  27.2   
                     2000-09-24 00:00:0

In [ ]:
# Grab with regular EC-SHIPS


basin_EC = 'north_atlantic'

RI_EC_pre = pd.read_csv('/Users/acheung/data/RI_Cases/RI_Events_'+basin_EC+'.csv')
if basin_EC == 'north_atlantic':
    EC_SHIPS_shear = pd.read_csv('/Users/acheung/data/EC_SHIPS/EC-SHIPS_deep_layer_shear_atl.csv')
elif basin_EC == 'east_pacific':
    EC_SHIPS_shear = pd.read_csv('/Users/acheung/data/EC_SHIPS/EC-SHIPS_deep_layer_shear_epac.csv')

RI_EC = RI_EC_pre.where(RI_EC_pre['Season'] >= 2016).dropna()
    
all_EC_shears = []
for i in range(len(RI_EC)):
    current_storm_shear_EC = EC_SHIPS_shear.where(EC_SHIPS_shear['Storm_ID'] ==
                                            RI_EC.iloc[[i]]['Storm_ID'].values[0]).dropna()
    if len(current_storm_shear_EC.where(
        current_storm_shear_EC['Time'] == RI_EC.iloc[[i]]['RI Start'].values[0]).dropna()['Deep_Layer_Shear']) == 0:
        # Some data is missing from EC-SHIPS, so we record NaN in these cases
        all_EC_shears.append(np.nan)
    else: # When data from EC-SHIPS is not missing
        current_shear_RI = current_storm_shear_EC.where(
            current_storm_shear_EC['Time'] == RI_EC.iloc[[i]]['RI Start'].values[0]).dropna()['Deep_Layer_Shear'].values[0]
        all_EC_shears.append(current_shear_RI)
    
RI_EC.insert(6, "Deep-Layer Shear (kt)", all_EC_shears)
RI_EC.to_csv("/Users/acheung/data/RI_Cases/RI_Events_with_EC_SHIPS_shear_"+basin_EC+".csv",index=False)
